In [33]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold 
from sksurv.util import Surv
from sklearn.impute import SimpleImputer
import scipy.stats as stats
from sksurv.ensemble import RandomSurvivalForest, GradientBoostingSurvivalAnalysis, ExtraSurvivalTrees
from sksurv.metrics import concordance_index_ipcw
from sklearn.utils.discovery import all_estimators
from tqdm import tqdm
from sksurv.linear_model import CoxPHSurvivalAnalysis, CoxnetSurvivalAnalysis, IPCRidge
from sksurv.tree import SurvivalTree, ExtraSurvivalTree
from sksurv.svm import MinlipSurvivalAnalysis
from sklearn.inspection import permutation_importance


# Data Loading

In [34]:
# Clinical data
df_clinical_train = pd.read_csv('data/X_train/clinical_train.csv')
df_clinical_test = pd.read_csv('data/X_test/clinical_test.csv')

# Molecular data
df_molecular_train = pd.read_csv('data/X_train/molecular_train.csv')
df_molecular_test = pd.read_csv('data/X_test/molecular_test.csv')

df_traget_train = pd.read_csv('data/target_train.csv')
df_traget_train['OS_YEARS'] = pd.to_numeric(df_traget_train['OS_YEARS'], errors='coerce')
df_traget_train['OS_STATUS'] = df_traget_train['OS_STATUS'].astype(bool)
df_traget_train.dropna(subset=['OS_STATUS','OS_YEARS'], inplace=True)

## A. Clinical data engineering

We engineer a feature by calculating ratios relative to WBC and defining specific disease burden indicators

In [35]:
def clinical_ratio(clinical_data):
    BM_BLAST = clinical_data['BM_BLAST']
    WBC = clinical_data['WBC']
    ANC = clinical_data['ANC']
    MONOCYTES = clinical_data['MONOCYTES']
    HB = clinical_data['HB']
    PLT = clinical_data['PLT']

    epsilon = 1e-6

    ANC_WBC = ANC / (WBC + epsilon)
    MONOCYTES_WBC = MONOCYTES / (WBC + epsilon)
    PLT_WBC = np.log1p(PLT / (WBC +epsilon)) # We use log1p because data or often asymetric
    HB_WBC = HB / (WBC + epsilon)

    BURDEN = WBC * (BM_BLAST/100)

    BLAST_20 = (BM_BLAST >= 20).astype(int).mask(BM_BLAST.isna())
    WBC_100 = (WBC >= 100).astype(int).mask(WBC.isna())

    return pd.DataFrame({
        "WBC_LOG": np.log1p(WBC),
        "ANC_WBC": ANC_WBC,
        "MONOCYTES_WBC": MONOCYTES_WBC,
        "PLT_WBC": PLT_WBC, 
        "HB_WBC": HB_WBC,
        "BURDEN": BURDEN,
        "BLAST_20": BLAST_20,
        "WBC_100": WBC_100
    })

df_clinical_ratio_train = clinical_ratio(df_clinical_train)
df_clinical_train = pd.concat([df_clinical_train, df_clinical_ratio_train], axis=1)
df_clinical_ratio_test = clinical_ratio(df_clinical_test)
df_clinical_test = pd.concat([df_clinical_test, df_clinical_ratio_test], axis=1)

We extract fundamental biological features by parsing the karyotype string to retrieve the number of chromosomes and encode the biological sex (XX vs XY)

In [36]:
def extract_sex_chromosome(cytogenetics):

    if pd.isna(cytogenetics):
        return None, None
    
    clone = cytogenetics.split(',')

    chromosomes = clone[0]
    try:
        chromosomes = int(chromosomes)
    except:
        chromosomes = None
    
    sex = None
    if len(clone)>1:
        match = re.search(r'(XX|XY)', clone[1], re.IGNORECASE)
        if match:
            sex = match.group(1).upper()
            if sex == 'XX':
                sex = 2
            elif sex == 'XY':
                sex = 1
            else:
                sex = 0
        
    return chromosomes, sex

df_clinical_train[['CHROMOSOMES', 'SEX']] = df_clinical_train['CYTOGENETICS'].apply(lambda x: pd.Series(extract_sex_chromosome(x))).replace({None: np.nan})
df_clinical_test[['CHROMOSOMES', 'SEX']] = df_clinical_test['CYTOGENETICS'].apply(lambda x: pd.Series(extract_sex_chromosome(x))).replace({None: np.nan})


We use regular expressions to parse the cytogenetic string and extract binary features for specific chromosomal abnormalities associated with favorable or adverse diagnosis (e.g., t(8;21), monosomy 7, complex karyotype)

In [37]:
def extract_specific_cytogenetics(cyto):

    if pd.isna(cyto):
         return pd.Series([0, 0, 0, 0, 0, 0, 0],
                            index=["CYTO_t8_21", "CYTO_inv16", "CYTO_t15_17", "CYTO_mono_5_7", 
                                   "CYTO_del_5q_7q", "CYTO_inv3", "CYTO_is_complex"])
    
    cytogenetic = str(cyto).lower()

    # t(8;21)
    t8_21 = 1 if re.search(r't\(8;21\)', cytogenetic) else 0
    # Inversion on 16
    inv16 = 1 if re.search(r'inv\(16\)|t\(16;16\)', cytogenetic) else 0
    # Leucimia promyélocytraire
    t15_17 = 1 if re.search(r't\(15;17\)', cytogenetic) else 0

    # Monosomi 5 or 7
    mono_5_7 = 1 if re.search(r'(^|[,\s])(-5|-7)([,\s]|$)', cytogenetic) else 0
    # del 5q or 7q
    del_5q_7q = 1 if re.search(r'del\((5|5q|7|7q)\)', cytogenetic) else 0
    # Inversion on 3
    inv3 = 1 if re.search(r'inv\(3\)|t\(3;3\)', cytogenetic) else 0

    is_complex = 1 if 'complex' in cytogenetic or cytogenetic.count(',') > 5 else 0 

    return pd.Series([t8_21, inv16, t15_17, mono_5_7, del_5q_7q, inv3, is_complex],
                     index=["CYTO_t8_21", "CYTO_inv16", "CYTO_t15_17", "CYTO_mono_5_7", "CYTO_del_5q_7q", "CYTO_inv3", "CYTO_is_complex"])

cyto_feature_train = df_clinical_train['CYTOGENETICS'].apply(extract_specific_cytogenetics)
df_clinical_train = pd.concat([df_clinical_train, cyto_feature_train], axis=1)

cyto_feature_test = df_clinical_test['CYTOGENETICS'].apply(extract_specific_cytogenetics)
df_clinical_test = pd.concat([df_clinical_test, cyto_feature_test], axis=1)


We parse the cytogenetics string to count the number of clones and specific types of mutations (deletions, duplications, translocations) for each patient

In [38]:
df_clinical_train['NUM_CLONE'] = df_clinical_train['CYTOGENETICS'].str.count(r'/') #Count the number of clone in the cytogenetics
df_clinical_test['NUM_CLONE'] = df_clinical_test['CYTOGENETICS'].str.count(r'/')

df_clinical_train['NUM_DELETION'] = df_clinical_train['CYTOGENETICS'].str.count(r'del\(')
df_clinical_test['NUM_DELETION'] = df_clinical_test['CYTOGENETICS'].str.count(r'del\(')

df_clinical_train['NUM_DUPLICATION'] = df_clinical_train['CYTOGENETICS'].str.count(r'dup\(')
df_clinical_test['NUM_DUPLICATION'] = df_clinical_test['CYTOGENETICS'].str.count(r'dup\(')

df_clinical_train['NUM_ISOCHROMOSOME'] = df_clinical_train['CYTOGENETICS'].str.count(r'i\(')
df_clinical_test['NUM_ISOCHROMOSOME'] = df_clinical_test['CYTOGENETICS'].str.count(r'i\(')

df_clinical_train['NUM_TRANSLOCATION'] = df_clinical_train['CYTOGENETICS'].str.count(r't\(')
df_clinical_test['NUM_TRANSLOCATION'] = df_clinical_test['CYTOGENETICS'].str.count(r't\(')

df_clinical_train['NUM_INSERTION'] = df_clinical_train['CYTOGENETICS'].str.count(r'ins\(')
df_clinical_test['NUM_INSERTION'] = df_clinical_test['CYTOGENETICS'].str.count(r'ins\(')

df_clinical_train['NUM_INVERSION'] = df_clinical_train['CYTOGENETICS'].str.count(r'inv\(')
df_clinical_test['NUM_INVERSION'] = df_clinical_test['CYTOGENETICS'].str.count(r'inv\(')


We parse the text to identify and count all unique chromosomes that present an anomaly (deletion, addition, or structural change)

In [39]:
def unique_chromosome(cyto):
    if pd.isna(cyto):
        return 0
    chromosome_in_paranthese = re.findall(r'\((\d{1,2})[p,q]?', cyto)
    chromosome_numeric = re.findall(r'[+-](\d{1,2})', cyto)
    all_chrom = list(map(int,chromosome_in_paranthese + chromosome_numeric))

    return len(set(all_chrom))

df_clinical_train['NUM_UNIQUE_CHROM'] = df_clinical_train['CYTOGENETICS'].apply(unique_chromosome)
df_clinical_test['NUM_UNIQUE_CHROM'] = df_clinical_test['CYTOGENETICS'].apply(unique_chromosome)

In [40]:
print("----CLINICAL TRAIN DATA----")
display(df_clinical_train.head())

print("----CLINICAL TEST DATA----")
display(df_clinical_test.head())

----CLINICAL TRAIN DATA----


,ID,CENTER,BM_BLAST,WBC,ANC,MONOCYTES,HB,PLT,CYTOGENETICS,WBC_LOG,...,CYTO_inv3,CYTO_is_complex,NUM_CLONE,NUM_DELETION,NUM_DUPLICATION,NUM_ISOCHROMOSOME,NUM_TRANSLOCATION,NUM_INSERTION,NUM_INVERSION,NUM_UNIQUE_CHROM
0,P132697,MSK,14.0,2.8,0.2,0.7,7.6,119.0,"46,xy,del(20)(q12)[2]/46,xy[18]",1.335001,...,0,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1
1,P132698,MSK,1.0,7.4,2.4,0.1,11.6,42.0,"46,xx",2.128232,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,P116889,MSK,15.0,3.7,2.1,0.1,14.2,81.0,"46,xy,t(3;3)(q25;q27)[8]/46,xy[12]",1.547563,...,1,0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1
3,P132699,MSK,1.0,3.9,1.9,0.1,8.9,77.0,"46,xy,del(3)(q26q27)[15]/46,xy[5]",1.589235,...,0,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1
4,P132700,MSK,6.0,128.0,9.7,0.9,11.1,195.0,"46,xx,t(3;9)(p13;q22)[10]/46,xx[10]",4.859812,...,0,0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1


----CLINICAL TEST DATA----


,ID,CENTER,BM_BLAST,WBC,ANC,MONOCYTES,HB,PLT,CYTOGENETICS,WBC_LOG,...,CYTO_inv3,CYTO_is_complex,NUM_CLONE,NUM_DELETION,NUM_DUPLICATION,NUM_ISOCHROMOSOME,NUM_TRANSLOCATION,NUM_INSERTION,NUM_INVERSION,NUM_UNIQUE_CHROM
0,KYW1,KYW,68.0,3.45,0.5865,NaN,7.6,48.0,"47,XY,+X,del(9)(q?)[15]/47,XY,+X[5]",1.492904,...,0,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1
1,KYW2,KYW,35.0,3.18,1.2402,NaN,10.0,32.0,"46,XY,der(3)?t(3;11)(q26.2;q23),add(4)(p15).de...",1.430311,...,0,0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,3
2,KYW3,KYW,NaN,12.40,8.6800,NaN,12.3,25.0,"47,XX,+8",2.595255,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,KYW4,KYW,61.0,5.55,2.0535,NaN,8.0,44.0,Normal,1.879465,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,KYW5,KYW,2.0,1.21,0.7381,NaN,8.6,27.0,"43,XY,dic(5;17)(q11.2;p11.2),-7,-13,-20,-22,+r...",0.792993,...,0,1,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5


## B. Molecular data engineering 

We group the molecular data by patient ID to calculate the total count of mutations

In [41]:
# We count the number of mutation
nmut_df_train = df_molecular_train.groupby('ID').size().reset_index()
nmut_df_train.columns = ['ID', 'NMUT']

nmut_df_test = df_molecular_test.groupby('ID').size().reset_index()
nmut_df_test.columns = ['ID', 'NMUT']


We aggregate the molecular data to count how many distinct genes carry mutations for each patient

In [42]:
# We also add a feature that will count the number of unique genes affected for each patient
df_uniquegene_train = df_molecular_train.groupby('ID')['GENE'].nunique().reset_index(name='UNIQUE_GENES')
df_uniquegene_test = df_molecular_test.groupby('ID')['GENE'].nunique().reset_index(name='UNIQUE_GENES')

df_molecular_train_modified = nmut_df_train.merge(df_uniquegene_train, on='ID', how='left').fillna({'UNIQUE_GENES':np.nan})
df_molecular_test_modified = nmut_df_test.merge(df_uniquegene_test, on='ID', how='left').fillna({'UNIQUE_GENES':np.nan})

We classify mutations into favorable or unfavorable categories and sum their contributions to create a global genetic risk score

In [43]:
def gene_classifier(gene_row):
    favorable_gene = ['NPM1', 'CEBPA', 'DDX41', 'SF3B1', 'ETV6', 'IDH1', 'IDH2']
    defavorable_gene = ['TP53', 'FLT3','ASXL1', 'RUNX1', 'WT1', 'NRAS', 'KRAS', 'KIT', 'PHF6', 
                        'SETBP1', 'PTEN', 'RB1', 'ATRX', 'BRAF', 'MYC', 'MLL', 'CDKN2A', 'CDK4', 'ABL1']

    if not isinstance(gene_row, str):
        return None

    if any(gene in gene_row for gene in favorable_gene):
        return -1
    if any(gene in gene_row for gene in defavorable_gene):
        return 1

    return 0

# Train
df_molecular_train['GENE_SCORE'] = df_molecular_train['GENE'].apply(lambda x: gene_classifier(x)).replace({None: np.nan})
df_gene_score_train = df_molecular_train.groupby('ID')['GENE_SCORE'].sum().reset_index()
df_molecular_train_modified = df_molecular_train_modified.merge(df_gene_score_train, on='ID', how='left').fillna({'GENE_SCORE':np.nan})


# Test
df_molecular_test['GENE_SCORE'] = df_molecular_test['GENE'].apply(lambda x: gene_classifier(x)).replace({None: np.nan})
df_gene_score_test = df_molecular_test.groupby('ID')['GENE_SCORE'].sum().reset_index()
df_molecular_test_modified = df_molecular_test_modified.merge(df_gene_score_test, on='ID', how='left').fillna({'GENE_SCORE':np.nan})

We apply one-hot encoding to the most frequent genes (minimum frequency of 15) to create a binary matrix representing the presence or absence of key driver mutations for each patient

In [44]:
def get_top_gene(data_df, min_frequency=15):
    gene_counts = data_df['GENE'].value_counts()
    top_genes = gene_counts[gene_counts >= min_frequency].index.tolist()
    return top_genes

top_genes = get_top_gene(df_molecular_train)

def create_gene_feature(molecular_df, selected_gene):
    subset_df = molecular_df[molecular_df['GENE'].isin(selected_gene)]

    # We pivot data : 
    # Index = ID of the patient
    # Column = Gene
    # Value = 1 if present (count), 0 if not
    gene_matrix = pd.crosstab(subset_df['ID'], subset_df['GENE'])

    # We only wont binary data
    gene_matrix = (gene_matrix > 0).astype(int)

    # If one of the top gene are not is the data, we add the column with only 0
    gene_matrix = gene_matrix.reindex(columns=selected_gene, fill_value=0)

    gene_matrix.columns = [f'GENE_{g}' for g in gene_matrix.columns]

    return gene_matrix

df_gene_feature_train = create_gene_feature(df_molecular_train, top_genes)
df_gene_feature_test = create_gene_feature(df_molecular_test, top_genes)

df_molecular_train_modified = df_molecular_train_modified.merge(df_gene_feature_train, on='ID', how='left')
df_molecular_test_modified = df_molecular_test_modified.merge(df_gene_feature_test, on='ID', how='left')

We isolate the VAF values for specific dangerous genes to provide the model with information about the intensity of these mutations (we only use the 3 most dangerous gene)

In [45]:
# We compute the VAF mean for each gene that are classified as risky 
defavorable_gene = ['TP53', 'FLT3', 'RUNX1']

df_vaf_gene_specific_train = pd.DataFrame(df_molecular_train['ID'].unique(), columns=['ID'])
df_vaf_gene_specific_test = pd.DataFrame(df_molecular_test['ID'].unique(), columns=['ID'])

for gene in defavorable_gene:
    vaf_gene_train_temp = df_molecular_train[df_molecular_train['GENE'] == gene].groupby('ID')['VAF'].mean().reset_index(name=f'VAF_{gene}')
    df_vaf_gene_specific_train = df_vaf_gene_specific_train.merge(vaf_gene_train_temp, on='ID', how='left').fillna({f'VAF_{gene}':0})

    vaf_gene_test_temp = df_molecular_test[df_molecular_test['GENE'] == gene].groupby('ID')['VAF'].mean().reset_index(name=f'VAF_{gene}')
    df_vaf_gene_specific_test = df_vaf_gene_specific_test.merge(vaf_gene_test_temp, on='ID', how='left').fillna({f'VAF_{gene}':0})

df_molecular_train_modified = df_molecular_train_modified.merge(df_vaf_gene_specific_train, on='ID', how='left')
df_molecular_test_modified = df_molecular_test_modified.merge(df_vaf_gene_specific_test, on='ID', how='left')

We aggregate the VAF data to extract the mean, maximum, and total sum of variant frequencies for each patient

In [46]:
df_vaf_stats_train = df_molecular_train.groupby('ID')['VAF'].agg(['mean', 'max', 'sum']).reset_index()
df_vaf_stats_train.columns = ['ID', 'MEAN_VAF', 'MAX_VAF', 'SUM_VAF']

df_vaf_stats_test = df_molecular_test.groupby('ID')['VAF'].agg(['mean', 'max', 'sum']).reset_index()
df_vaf_stats_test.columns = ['ID', 'MEAN_VAF', 'MAX_VAF', 'SUM_VAF']

df_molecular_train_modified = df_molecular_train_modified.merge(df_vaf_stats_train, on='ID', how='left').fillna({'MEAN_VAF':np.nan, 'MAX_VAF':np.nan, 'SUM_VAF':np.nan})
df_molecular_test_modified = df_molecular_test_modified.merge(df_vaf_stats_test, on='ID', how='left').fillna({'MEAN_VAF':np.nan, 'MAX_VAF':np.nan, 'SUM_VAF':np.nan})

We compute the ratios of different mutation effects (such as frameshift or stop-gained variants) relative to the total count of mutations to assess the severity of genetic alterations

In [47]:
def ratio_frameshift_variant(effect_data):
    num_mutation = effect_data.shape[0]
    if num_mutation == 0:
        return 0
    else:
        num_framsehift_variant = (effect_data == 'frameshift_variant').sum()
        ratio = num_framsehift_variant / num_mutation
        return ratio

def ratio_stop_gained(effect_data):
    num_mutation = effect_data.shape[0]
    if num_mutation == 0:
        return 0
    else:
        num_stop_gained = (effect_data == 'stop_gained').sum()
        ratio = num_stop_gained / num_mutation
        return ratio

def ratio_stop_lost(effect_data):
    num_mutation = effect_data.shape[0]
    if num_mutation == 0:
        return 0
    else:
        num_stop_lost = (effect_data == 'stop_lost').sum()
        ratio = num_stop_lost / num_mutation
        return ratio

def ratio_splice_site_variant(effect_data):
    num_mutation = effect_data.shape[0]
    if num_mutation == 0:
        return 0
    else:
        num_ss_variant = (effect_data == 'splice_site_variant').sum()
        ratio = num_ss_variant / num_mutation
        return ratio

df_frameshit_variant_effect_train = df_molecular_train.groupby('ID')['EFFECT'].apply(ratio_frameshift_variant).reset_index(name='RATIO_FRAMESHIFT_VARIANT')
df_frameshit_variant_effect_test = df_molecular_test.groupby('ID')['EFFECT'].apply(ratio_frameshift_variant).reset_index(name='RATIO_FRAMESHIFT_VARIANT')

df_stop_gained_effect_train = df_molecular_train.groupby('ID')['EFFECT'].apply(ratio_stop_gained).reset_index(name='RATIO_STOP_GAINED')
df_stop_gained_effect_test = df_molecular_test.groupby('ID')['EFFECT'].apply(ratio_stop_gained).reset_index(name='RATIO_STOP_GAINED')

df_stop_lost_effect_train = df_molecular_train.groupby('ID')['EFFECT'].apply(ratio_stop_lost).reset_index(name='RATIO_STOP_LOST')
df_stop_lost_effect_test = df_molecular_test.groupby('ID')['EFFECT'].apply(ratio_stop_lost).reset_index(name='RATIO_STOP_LOST')

df_splice_site_variant_effect_train = df_molecular_train.groupby('ID')['EFFECT'].apply(ratio_splice_site_variant).reset_index(name='RATIO_SPLICE_SITE_VARIANT')
df_splice_site_variant_effect_test = df_molecular_test.groupby('ID')['EFFECT'].apply(ratio_splice_site_variant).reset_index(name='RATIO_SPLICE_SITE_VARIANT')

df_effect_train = (
    df_frameshit_variant_effect_train
    .merge(df_stop_gained_effect_train, on='ID', how='left')
    .merge(df_stop_lost_effect_train, on='ID', how='left')
    .merge(df_splice_site_variant_effect_train, on='ID', how='left')
)

df_effect_test = (
    df_frameshit_variant_effect_test
    .merge(df_stop_gained_effect_test, on='ID', how='left')
    .merge(df_stop_lost_effect_test, on='ID', how='left')
    .merge(df_splice_site_variant_effect_test, on='ID', how='left')
)


df_molecular_train_modified = df_molecular_train_modified.merge(df_effect_train, on='ID', how='left')
df_molecular_test_modified = df_molecular_test_modified.merge(df_effect_test, on='ID', how='left')


We map categorical mutation effects to a numerical severity scale (0 for neutral, 2 for very bad) and calculate summary statistics to represent the overall biological damage for each patient

In [48]:
effect_score = {
    # Very Bad
    'frameshift_variant': 2,
    'stop_gained': 2,
    'stop_lost': 2,
    'splice_site_variant': 2,
    'PTD': 2,
    'ITD': 2,

    # Bad
    'non_synonymous_codon': 1,
    'inframe_codon_loss': 1,
    'inframe_codon_gain': 1,
    'inframe_variant': 1,
    'initiator_codon_change': 1,
    'complex_change_in_transcript': 1,

    # Neutral
    'synonymous_codon': 0,
    'stop_retained_variant': 0,
    '3_prime_UTR_variant': 0,
    '2KB_upstream_variant': 0
}

df_molecular_train['EFFECT_SCORE'] = df_molecular_train['EFFECT'].map(effect_score).fillna(0)
effect_score_df_train = df_molecular_train.groupby('ID')['EFFECT_SCORE'].agg(['mean', 'max', 'sum']).reset_index()
effect_score_df_train.columns = ['ID', 'MEAN_EFFECT', 'MAX_EFFECT', 'SUM_EFFECT']

df_molecular_test['EFFECT_SCORE'] = df_molecular_test['EFFECT'].map(effect_score).fillna(0)
effect_score_df_test = df_molecular_test.groupby('ID')['EFFECT_SCORE'].agg(['mean', 'max', 'sum']).reset_index()
effect_score_df_test.columns = ['ID', 'MEAN_EFFECT', 'MAX_EFFECT', 'SUM_EFFECT']

df_molecular_train_modified = (
    df_molecular_train_modified
    .merge(effect_score_df_train, on='ID', how='left').fillna({'MEAN_EFFECT':np.nan, 'MAX_EFFECT':np.nan, 'SUM_EFFECT':np.nan})
)

df_molecular_test_modified = (
    df_molecular_test_modified
    .merge(effect_score_df_test, on='ID', how='left').fillna({'MEAN_EFFECT':np.nan, 'MAX_EFFECT':np.nan, 'SUM_EFFECT':np.nan})
)


We identify common mutation effects and create binary features to flag their presence for each patient

In [49]:
def get_common_effect(data_df, min_frequency=10):
    effect_counts = data_df['EFFECT'].value_counts()
    common_effect = effect_counts[effect_counts >= min_frequency].index.tolist()
    return common_effect

def create_binary_effect_features(data_df, common_effect):

    all_ids = data_df['ID'].unique()
    df_effect_features = pd.DataFrame({'ID': all_ids})

    for effect in common_effect:
        feature_name = f'HAS_{effect.upper()}'
        ids_with_effect = data_df[data_df['EFFECT'] == effect]['ID'].unique()
        df_effect_features[feature_name] = df_effect_features['ID'].isin(ids_with_effect).astype(int)
    return df_effect_features

critical_effect = get_common_effect(df_molecular_train)
df_binary_effect_features_train = create_binary_effect_features(df_molecular_train, critical_effect)
df_binary_effect_features_test = create_binary_effect_features(df_molecular_test, critical_effect)

df_molecular_train_modified = df_molecular_train_modified.merge(df_binary_effect_features_train, on='ID', how='left')
df_molecular_test_modified = df_molecular_test_modified.merge(df_binary_effect_features_test, on='ID', how='left')

In [50]:
print("----MOLECULAR TRAIN DATA----")
display(df_molecular_train_modified.head())
print("----MOLECULAR TEST DATA----")
display(df_molecular_test_modified.head())


----MOLECULAR TRAIN DATA----


,ID,NMUT,UNIQUE_GENES,GENE_SCORE,GENE_TET2,GENE_ASXL1,GENE_SF3B1,GENE_DNMT3A,GENE_RUNX1,GENE_SRSF2,...,HAS_FRAMESHIFT_VARIANT,HAS_STOP_GAINED,HAS_SPLICE_SITE_VARIANT,HAS_INFRAME_CODON_LOSS,HAS_PTD,HAS_INFRAME_CODON_GAIN,HAS_ITD,HAS_INITIATOR_CODON_CHANGE,HAS_2KB_UPSTREAM_VARIANT,HAS_COMPLEX_CHANGE_IN_TRANSCRIPT
0,P100000,6,6,0,1.0,0.0,0.0,1.0,0.0,0.0,...,1,1,1,0,0,0,0,0,0,0
1,P100001,2,2,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0
2,P100002,2,2,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
3,P100004,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
4,P100006,5,5,0,1.0,0.0,0.0,0.0,0.0,1.0,...,1,1,0,0,0,0,0,0,0,0


----MOLECULAR TEST DATA----


,ID,NMUT,UNIQUE_GENES,GENE_SCORE,GENE_TET2,GENE_ASXL1,GENE_SF3B1,GENE_DNMT3A,GENE_RUNX1,GENE_SRSF2,...,HAS_FRAMESHIFT_VARIANT,HAS_STOP_GAINED,HAS_SPLICE_SITE_VARIANT,HAS_INFRAME_CODON_LOSS,HAS_PTD,HAS_INFRAME_CODON_GAIN,HAS_ITD,HAS_INITIATOR_CODON_CHANGE,HAS_2KB_UPSTREAM_VARIANT,HAS_COMPLEX_CHANGE_IN_TRANSCRIPT
0,KYW1,4,4,0,0.0,0.0,0.0,1.0,0.0,0.0,...,1,1,0,0,0,0,1,0,0,0
1,KYW10,2,2,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
2,KYW100,2,2,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
3,KYW1000,5,3,0,1.0,0.0,0.0,0.0,0.0,0.0,...,1,1,0,0,0,0,0,0,0,0
4,KYW1001,6,6,2,0.0,1.0,0.0,0.0,1.0,1.0,...,1,1,0,0,0,0,0,0,0,0


# Running with different model

In [51]:
x_clinical_train = df_clinical_train
x_molecular_train = df_molecular_train_modified
x_train = pd.merge(x_clinical_train,x_molecular_train, on='ID', how='left')

cols_to_exclude = ['ID', 'CENTER', 'CYTOGENETICS', 'GENE', 'EFFECT']
features = [c for c in x_train.columns if c not in cols_to_exclude and pd.api.types.is_numeric_dtype(x_train[c])]


A. GradientBoostingSurvivalAnalysis

In [52]:
X = x_train.loc[x_train['ID'].isin(df_traget_train['ID']), features]
y = Surv.from_dataframe('OS_STATUS', 'OS_YEARS', df_traget_train)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.125, random_state=42)

imputer = SimpleImputer(strategy='median')
X_train[features] = imputer.fit_transform(X_train[features])
X_test[features] = imputer.transform(X_test[features])

results_GBSA = []
model_GBSA = GradientBoostingSurvivalAnalysis(loss='coxph', n_estimators=200, random_state=42)

model_GBSA.fit(X_train, y_train)
cindex_train = concordance_index_ipcw(y_train, y_train, model_GBSA.predict(X_train), tau=7)[0]
cindex_test = concordance_index_ipcw(y_train, y_test, model_GBSA.predict(X_test), tau=7)[0]    

results_GBSA.append(['GBSA', cindex_train, cindex_test])
df_results_GBSA = pd.DataFrame(results_GBSA, columns=['Regressor', 'Cox Index Train', 'Cox Index Test'])

display(df_results_GBSA)

,Regressor,Cox Index Train,Cox Index Test
0,GBSA,0.774003,0.723111


We run again the model but we do it without all features that have a negative permutation importance score

In [53]:
perm_imp_GBSA = permutation_importance(model_GBSA, X_test, y_test, n_repeats=1, random_state=42)
importance_score_GBSA = perm_imp_GBSA.importances_mean
feature_name_GBSA = X_test.columns

feature_importance_GBSA = pd.DataFrame({
    'Feature': feature_name_GBSA,
    'Importance': importance_score_GBSA
})

feature_importance_GBSA = feature_importance_GBSA.sort_values(by='Importance', ascending=False)
neg_feature_GBSA = feature_importance_GBSA[feature_importance_GBSA['Importance']<0]['Feature']

new_features_GBSA = [f for f in features if f not in list(neg_feature_GBSA)]

X = x_train.loc[x_train['ID'].isin(df_traget_train['ID']), new_features_GBSA]
y = Surv.from_dataframe('OS_STATUS', 'OS_YEARS', df_traget_train)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.125, random_state=42)

imputer = SimpleImputer(strategy='median')
X_train[new_features_GBSA] = imputer.fit_transform(X_train[new_features_GBSA])
X_test[new_features_GBSA] = imputer.transform(X_test[new_features_GBSA])

results_GBSA = []
model_GBSA = GradientBoostingSurvivalAnalysis(loss='coxph', n_estimators=200, random_state=42)

model_GBSA.fit(X_train, y_train)
cindex_train = concordance_index_ipcw(y_train, y_train, model_GBSA.predict(X_train), tau=7)[0]
cindex_test = concordance_index_ipcw(y_train, y_test, model_GBSA.predict(X_test), tau=7)[0]    

results_GBSA.append(['GBSA', cindex_train, cindex_test])

df_results_GBSA = pd.DataFrame(results_GBSA, columns=['Regressor', 'Cox Index Train', 'Cox Index Test'])
display(df_results_GBSA)


,Regressor,Cox Index Train,Cox Index Test
0,GBSA,0.774206,0.732911


In [54]:
model_GBSA_optimization = GradientBoostingSurvivalAnalysis(loss='coxph', random_state=42)

params_GBSA = {
    "n_estimators": stats.randint(100, 800),
    "learning_rate": stats.uniform(0.01, 0.1),
    "max_depth": stats.randint(3, 7),
    "min_samples_leaf": stats.randint(10,50),
    "max_features": ['sqrt', 'log2', None],
    "subsample": stats.uniform(0.7, 0.2)
}

cv_GBSA = KFold(n_splits=2, shuffle=True, random_state=42)

search_GBSA = RandomizedSearchCV(
    estimator=model_GBSA_optimization,
    param_distributions=params_GBSA,
    n_iter=5, # My PC is very slow we only use 2 here but the best would be to us 50 or even 100
    cv=cv_GBSA,
    verbose=1
)

search_GBSA.fit(X_train, y_train)
best_model_GBSA = search_GBSA.best_estimator_
score_test_GBSA = best_model_GBSA.predict(X_test)
c_index_ipcw_GBSA = concordance_index_ipcw(y_train, y_test, score_test_GBSA, tau=7)[0]

print("Best params: ", search_GBSA.best_params_)
print("Score CV: ", search_GBSA.best_score_)
print("Score IPCW Score index ", c_index_ipcw_GBSA)

Fitting 2 folds for each of 5 candidates, totalling 10 fits
Best params:  {'learning_rate': 0.03284055546314148, 'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 42, 'n_estimators': 584, 'subsample': 0.8002200360722076}
Score CV:  0.7280775657832169
Score IPCW Score index  0.7323088016314836


In [55]:
x_clinical_test = df_clinical_test
x_molecular_test = df_molecular_test_modified
x_test = pd.merge(x_clinical_test,x_molecular_test, on='ID', how='left')

x_test[new_features_GBSA] = imputer.transform(x_test[new_features_GBSA])

prediction_on_test_set_GBSA = model_GBSA.predict(x_test.loc[:,new_features_GBSA])
prediction_on_test_set_GBSA_optimized = best_model_GBSA.predict(x_test.loc[:,new_features_GBSA])

submission_GBSA = pd.Series(prediction_on_test_set_GBSA, index=x_test['ID'])
submission_GBSA_optimized = pd.Series(prediction_on_test_set_GBSA_optimized, index=x_test['ID'])

submission_normalized_GBSA = (submission_GBSA - submission_GBSA.min()) / (submission_GBSA.max() - submission_GBSA.min())
submission_normalized_GBSA = pd.DataFrame(submission_normalized_GBSA, columns=['risk_score'])
submission_normalized_GBSA.to_csv('result_GBSA.csv')
print(f"C-Index: {cindex_test}, model saved")

submission_normalized_GBSA_optimized = (submission_GBSA_optimized - submission_GBSA_optimized.min()) / (submission_GBSA_optimized.max() - submission_GBSA_optimized.min())
submission_normalized_GBSA_optimized = pd.DataFrame(submission_normalized_GBSA_optimized, columns=['risk_score'])
submission_normalized_GBSA_optimized.to_csv('result_GBSA_optimized.csv')
print(f"C-Index: {c_index_ipcw_GBSA}, model optimized saved")


C-Index: 0.7329110981196992, model saved
C-Index: 0.7323088016314836, model optimized saved


B. ExtraSurvivalTrees

In [56]:
X = x_train.loc[x_train['ID'].isin(df_traget_train['ID']), features]
y = Surv.from_dataframe('OS_STATUS', 'OS_YEARS', df_traget_train)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.125, random_state=42)

imputer = SimpleImputer(strategy='median')
X_train[features] = imputer.fit_transform(X_train[features])
X_test[features] = imputer.transform(X_test[features])

results_EST = []
model_EST = ExtraSurvivalTrees(n_estimators=200, random_state=42)

model_EST.fit(X_train, y_train)
cindex_train = concordance_index_ipcw(y_train, y_train, model_EST.predict(X_train), tau=7)[0]
cindex_test = concordance_index_ipcw(y_train, y_test, model_EST.predict(X_test), tau=7)[0]    

results_EST.append(['ExtraSurvivalTrees', cindex_train, cindex_test])

df_results_EST = pd.DataFrame(results_EST, columns=['Regressor', 'Cox Index Train', 'Cox Index Test']).sort_values(by='Cox Index Test', ascending=False)
display(df_results_EST)

,Regressor,Cox Index Train,Cox Index Test
0,ExtraSurvivalTrees,0.811637,0.713749


In [57]:
perm_imp_EST = permutation_importance(model_EST, X_test, y_test, n_repeats=1, random_state=42)
importance_score_EST = perm_imp_EST.importances_mean
feature_name_EST = X_test.columns

feature_importance_EST = pd.DataFrame({
    'Feature': feature_name_EST,
    'Importance': importance_score_EST
})

feature_importance_EST = feature_importance_EST.sort_values(by='Importance', ascending=False)
neg_feature_EST = feature_importance_EST[feature_importance_EST['Importance']<0]['Feature']

new_features_EST = [f for f in features if f not in list(neg_feature_EST)]
X = x_train.loc[x_train['ID'].isin(df_traget_train['ID']), new_features_EST]
y = Surv.from_dataframe('OS_STATUS', 'OS_YEARS', df_traget_train)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.125, random_state=42)

imputer = SimpleImputer(strategy='median')
X_train[new_features_EST] = imputer.fit_transform(X_train[new_features_EST])
X_test[new_features_EST] = imputer.transform(X_test[new_features_EST])

results_EST = []
model_EST = ExtraSurvivalTrees(n_estimators=200, random_state=42)

model_EST.fit(X_train, y_train)
cindex_train = concordance_index_ipcw(y_train, y_train, model_EST.predict(X_train), tau=7)[0]
cindex_test = concordance_index_ipcw(y_train, y_test, model_EST.predict(X_test), tau=7)[0]    

results_EST.append(['ExtraSurvivalTrees', cindex_train, cindex_test])

df_results_EST = pd.DataFrame(results_EST, columns=['Regressor', 'Cox Index Train', 'Cox Index Test'])
display(df_results_EST)


,Regressor,Cox Index Train,Cox Index Test
0,ExtraSurvivalTrees,0.801671,0.734808


In [58]:
model_EST_optimization = ExtraSurvivalTrees(random_state=42)

params_EST = {
    "n_estimators": stats.randint(300, 800),
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": stats.randint(4, 20),
    "min_samples_leaf": stats.randint(2, 10),
    "max_features": ['sqrt', 'log2', 0.5, 0.7, 0.9],
    "bootstrap": [True, False]
}

cv_RSF = KFold(n_splits=2, shuffle=True, random_state=42)

search_EST = RandomizedSearchCV(
    estimator=model_EST_optimization,
    param_distributions=params_EST,
    n_iter=5, # My PC is very slow we only use 2 here but the best would be to us 50 or even 100
    cv=cv_RSF,
    verbose=1
)

search_EST.fit(X_train, y_train)
best_model_EST = search_EST.best_estimator_
score_test_EST = best_model_EST.predict(X_test)
c_index_ipcw_EST = concordance_index_ipcw(y_train, y_test, score_test_EST, tau=7)[0]

print("Best params: ", search_EST.best_params_)
print("Score CV: ", search_EST.best_score_)
print("Score IPCW Score index ", c_index_ipcw_EST)


Fitting 2 folds for each of 5 candidates, totalling 10 fits
Best params:  {'bootstrap': True, 'max_depth': 20, 'max_features': 0.9, 'min_samples_leaf': 8, 'min_samples_split': 17, 'n_estimators': 607}
Score CV:  0.7183537431222389
Score IPCW Score index  0.7318988591661438


In [59]:
x_clinical_test = df_clinical_test
x_molecular_test = df_molecular_test_modified
x_test = pd.merge(x_clinical_test,x_molecular_test, on='ID', how='left')

x_test[new_features_EST] = imputer.transform(x_test[new_features_EST])

prediction_on_test_set_EST = model_EST.predict(x_test.loc[:,new_features_EST])
prediction_on_test_set_EST_optimized = best_model_EST.predict(x_test.loc[:,new_features_EST])

submission_EST = pd.Series(prediction_on_test_set_EST, index=x_test['ID'])
submission_EST_optimized = pd.Series(prediction_on_test_set_EST_optimized, index=x_test['ID'])

submission_normalized_EST = (submission_EST - submission_EST.min()) / (submission_EST.max() - submission_EST.min())
submission_normalized_EST = pd.DataFrame(submission_normalized_EST, columns=['risk_score'])
submission_normalized_EST.to_csv('result_EST.csv')
print(f"C-Index: {cindex_test}, model saved")

submission_normalized_EST_optimized = (submission_EST_optimized - submission_EST_optimized.min()) / (submission_EST_optimized.max() - submission_EST_optimized.min())
submission_normalized_EST_optimized = pd.DataFrame(submission_normalized_EST_optimized, columns=['risk_score'])
submission_normalized_EST_optimized.to_csv('result_EST_optimized.csv')
print(f"C-Index: {c_index_ipcw_EST}, model optimized saved")


C-Index: 0.7348078282986609, model saved
C-Index: 0.7318988591661438, model optimized saved


C. RandomSurvivalForest

In [60]:
X = x_train.loc[x_train['ID'].isin(df_traget_train['ID']), features]
y = Surv.from_dataframe('OS_STATUS', 'OS_YEARS', df_traget_train)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.125, random_state=42)

imputer = SimpleImputer(strategy='median')
X_train[features] = imputer.fit_transform(X_train[features])
X_test[features] = imputer.transform(X_test[features])

results_RSF = []
model_RSF = RandomSurvivalForest(n_estimators=200, random_state=42)

model_RSF.fit(X_train, y_train)
cindex_train = concordance_index_ipcw(y_train, y_train, model_RSF.predict(X_train), tau=7)[0]
cindex_test = concordance_index_ipcw(y_train, y_test, model_RSF.predict(X_test), tau=7)[0]    

results_RSF.append(['RSF', cindex_train, cindex_test])

df_results_RSF = pd.DataFrame(results_RSF, columns=['Regressor', 'Cox Index Train', 'Cox Index Test']).sort_values(by='Cox Index Test', ascending=False)
display(df_results_RSF)

,Regressor,Cox Index Train,Cox Index Test
0,RSF,0.875867,0.731461


In [61]:
perm_imp_RSF = permutation_importance(model_RSF, X_test, y_test, n_repeats=1, random_state=42)
importance_score_RSF = perm_imp_RSF.importances_mean
feature_name_RSF = X_test.columns

feature_importance_RSF = pd.DataFrame({
    'Feature': feature_name_RSF,
    'Importance': importance_score_RSF
})

feature_importance_RSF = feature_importance_RSF.sort_values(by='Importance', ascending=False)
neg_feature_RSF = feature_importance_RSF[feature_importance_RSF['Importance']<0]['Feature']

new_features_RSF = [f for f in features if f not in list(neg_feature_RSF)]
X = x_train.loc[x_train['ID'].isin(df_traget_train['ID']), new_features_RSF]
y = Surv.from_dataframe('OS_STATUS', 'OS_YEARS', df_traget_train)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.125, random_state=42)

imputer = SimpleImputer(strategy='median')
X_train[new_features_RSF] = imputer.fit_transform(X_train[new_features_RSF])
X_test[new_features_RSF] = imputer.transform(X_test[new_features_RSF])

results_RSF = []
model_RSF = RandomSurvivalForest(n_estimators=200, random_state=42)

model_RSF.fit(X_train, y_train)
cindex_train = concordance_index_ipcw(y_train, y_train, model_RSF.predict(X_train), tau=7)[0]
cindex_test = concordance_index_ipcw(y_train, y_test, model_RSF.predict(X_test), tau=7)[0]    

results_RSF.append(['RSF', cindex_train, cindex_test])

df_results_RSF = pd.DataFrame(results_RSF, columns=['Regressor', 'Cox Index Train', 'Cox Index Test'])
display(df_results_RSF)

,Regressor,Cox Index Train,Cox Index Test
0,RSF,0.861821,0.740343


In [62]:
model_RSF_optimization = RandomSurvivalForest(random_state=42)

params_RSF = {
    "n_estimators": stats.randint(200, 600),
    "max_depth": [None, 10, 20, 30, 50],
    "min_samples_split": stats.randint(4, 20),
    "min_samples_leaf": stats.randint(2, 10),
    "max_features": ['sqrt', 'log2', 0.5, 0.7],
    "bootstrap": [True, False]
}

cv_RSF = KFold(n_splits=2, shuffle=True, random_state=42)

search_RSF = RandomizedSearchCV(
    estimator=model_RSF_optimization,
    param_distributions=params_RSF,
    n_iter=5, # My PC is very slow we only use 2 here but the best would be to us 50 or even 100
    cv=cv_RSF,
    verbose=1
)

search_RSF.fit(X_train, y_train)
best_model_RSF = search_RSF.best_estimator_
score_test_RSF = best_model_RSF.predict(X_test)
c_index_ipcw_RSF = concordance_index_ipcw(y_train, y_test, score_test_RSF, tau=7)[0]

print("Best params: ", search_RSF.best_params_)
print("Score CV: ", search_RSF.best_score_)
print("Score IPCW Score index ", c_index_ipcw_RSF)


Fitting 2 folds for each of 5 candidates, totalling 10 fits
Best params:  {'bootstrap': False, 'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'min_samples_split': 13, 'n_estimators': 586}
Score CV:  0.7207565452479878
Score IPCW Score index  0.7368670794121454


In [63]:
x_clinical_test = df_clinical_test
x_molecular_test = df_molecular_test_modified
x_test = pd.merge(x_clinical_test,x_molecular_test, on='ID', how='left')

x_test[new_features_RSF] = imputer.transform(x_test[new_features_RSF])

prediction_on_test_set_RSF = model_RSF.predict(x_test.loc[:,new_features_RSF])
prediction_on_test_set_RSF_optimized = best_model_RSF.predict(x_test.loc[:,new_features_RSF])

submission_RSF = pd.Series(prediction_on_test_set_RSF, index=x_test['ID'])
submission_RSF_optimized = pd.Series(prediction_on_test_set_RSF_optimized, index=x_test['ID'])

submission_normalized_RSF = (submission_RSF - submission_RSF.min()) / (submission_RSF.max() - submission_RSF.min())
submission_normalized_RSF = pd.DataFrame(submission_normalized_RSF, columns=['risk_score'])
submission_normalized_RSF.to_csv('result_RSF.csv')
print(f"C-Index: {cindex_test}, model saved")

submission_normalized_RSF_optimized = (submission_RSF_optimized - submission_RSF_optimized.min()) / (submission_RSF_optimized.max() - submission_RSF_optimized.min())
submission_normalized_RSF_optimized = pd.DataFrame(submission_normalized_RSF_optimized, columns=['risk_score'])
submission_normalized_RSF_optimized.to_csv('result_RSF_optimized.csv')
print(f"C-Index: {c_index_ipcw_RSF}, model optimized saved")


C-Index: 0.7403432078630965, model saved
C-Index: 0.7368670794121454, model optimized saved
